# Lab 9 - Summarizing Attendance and Practice Attempts

In the last lab, you combined the information for attendance and practice quizzes into one combined table.  In this lab, you will transform these data into two summary tables.

1. For each class, you will create a table that summarizes the attendance of all students in that class.
2. For each class that has practice quizzes, you will create a table that summarized the number of attempts, overall and for each module.

## Tasks

To complete this lab, complete each of the following tasks.

#### Task 1 - Attendance Summary

For each of the classes contained in `attendance_example.zip`, we want to create a table with three columns: `FirstName`, `LastName`, `Attendance`.  Note that `Attendance` will represent the maximum number of attempts on the quiz, which corresponds to that students number of days they have attended.  To complete this task, do the following.

1. Write a helper function that takes a class identifier and the overall dataframe as arguments; and returns the table described above.
2. Use a loop and your helper functions to create the attendance quiz table for each class and then write the contents to a csv file.

In [1]:
import pandas as pd
from dfply import *

In [2]:
df = pd.read_csv("attendance_practice_attempts.csv")
df.head()

,Org Defined ID,UserName,FirstName,LastName,Attempt #,Score,Out Of,Attempt_Start,Attempt_End,Percent,Class,Category,Module
0,14460432,au9747cp,Jericho,Greer,1,1,1,2019-01-14 14:00:00,2019-01-14 14:06:00,100 %,dsci494,Attendance,NaN
1,14460432,au9747cp,Jericho,Greer,2,1,1,2019-01-16 14:00:00,2019-01-16 14:08:00,100 %,dsci494,Attendance,NaN
2,14460432,au9747cp,Jericho,Greer,3,1,1,2019-01-18 14:00:00,2019-01-18 14:05:00,100 %,dsci494,Attendance,NaN
3,14460432,au9747cp,Jericho,Greer,4,1,1,2019-01-23 14:00:00,2019-01-23 14:06:00,100 %,dsci494,Attendance,NaN
4,14460432,au9747cp,Jericho,Greer,5,1,1,2019-01-25 14:00:00,2019-01-25 14:10:00,100 %,dsci494,Attendance,NaN


In [3]:
from glob import glob
files = glob('./data/attendance_example/*/*.csv')
files

['./data/attendance_example/dsci494s7/Attendance Quiz - User Attempts.csv',
 './data/attendance_example/dsci494s7/dsci494attendance.csv',
 './data/attendance_example/dsci494s7/dsci494practice.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 1 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 2 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 3 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 4 - User Attempts.csv',
 './data/attendance_example/stat180s18/Attendance Quiz - User Attempts.csv',
 './data/attendance_example/stat180s18/stat180attendance.csv',
 './data/attendance_example/stat491s1/Attendance Quiz - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 1 - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 2 - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 3 - User Attemp

In [4]:
def attentbl (df, classid): 
    return (df 
                >> select(X.FirstName, X.LastName, X['Attempt #'], X.Class)
                >> filter_by(X.Class == classid)
                >> group_by(X.Class, X.FirstName, X.LastName) 
                >> summarize(Attendance = X['Attempt #'].max())
                #>> drop(X.Class)
            )
attentbl(df, 'dsci494')

,LastName,FirstName,Class,Attendance
0,Aug,Ben,dsci494,19
1,Montmartre,Christendom,dsci494,18
2,Bali,Dickerson,dsci494,19
3,Cargill,Goodyear,dsci494,19
4,Greer,Jericho,dsci494,18
5,Harbin,Mansfield,dsci494,18
6,Hadrian,Merrimack,dsci494,14
7,Gaylord,Philippine,dsci494,19
8,Galbreath,Ross,dsci494,19
9,Kafka,Willard,dsci494,21


In [5]:
import re
CLASS_NAME_RE = re.compile(r'^\./data/attendance_example/([a-z]*[0-9]*).*\.csv$')
class_name = lambda p: CLASS_NAME_RE.match(p).group(1) 
class_names = lambda files: [class_name(p) for p in files]
class_names(files)

['dsci494',
 'dsci494',
 'dsci494',
 'dsci494',
 'dsci494',
 'dsci494',
 'dsci494',
 'stat180',
 'stat180',
 'stat491',
 'stat491',
 'stat491',
 'stat491',
 'stat491',
 'stat491',
 'stat491']

In [6]:
col_names = ['LastName', 'FirstName', 'Class','Attendance']
attendancequiz = pd.DataFrame(columns=col_names)
for cls in class_names(files):
    attendancequiz = attendancequiz >> union(attentbl(df, cls)) 

In [7]:
attendancequiz >> drop(X.Class) >> head

,LastName,FirstName,Attendance
0,Aug,Ben,19
1,Montmartre,Christendom,18
2,Bali,Dickerson,19
3,Cargill,Goodyear,19
4,Greer,Jericho,18


In [19]:
dsci494attendance = attendancequiz >> filter_by(X.Class == 'dsci494') >> drop(X.Class)
dsci494attendance.to_csv('./data/attendance_example/dsci494s7/dsci494attendance.csv', index=False)

In [20]:
dsci494attendance

,LastName,FirstName,Attendance
0,Aug,Ben,19
1,Montmartre,Christendom,18
2,Bali,Dickerson,19
3,Cargill,Goodyear,19
4,Greer,Jericho,18
5,Harbin,Mansfield,18
6,Hadrian,Merrimack,14
7,Gaylord,Philippine,19
8,Galbreath,Ross,19
9,Kafka,Willard,21


In [9]:
stat491attendance = attendancequiz >> filter_by(X.Class == 'stat491') >> drop(X.Class)
stat491attendance.to_csv('./data/attendance_example/stat491s1/stat491attendace.csv', index=False)

In [10]:
stat180attendance = attendancequiz >> filter_by(X.Class == 'stat180') >> drop(X.Class)
stat180attendance.to_csv('./data/attendance_example/stat180s18/stat180attendance.csv', index=False)

#### Task 2 - Practice Quiz Summary

Some of the classes contained in `attendance_example.zip` contain information about attempts on practice quizzes for four modules.  We want to create a table for each class that summarizes the practice quiz attempts.  This table should contain the following columns: `FirstName`, `LastName`, `Module 1`, `Module 2`, `Module 3`, `Module 4`, and `Total Attempts`.  Note that, for example, `Module 1` contains the total number of attempts each student made on the corresponding quiz and `Total Attempts` contains the total number of attempts on all four quizzes.


1. Create a list of class that have practice quizzes.
2. Write a helper function that takes a class identifier and the overall dataframe as arguments; and returns the table described above.
3. Use a loop and your helper functions to create the practice quiz table for each class and then write the contents to a csv file.

In [11]:
practice_class = ['dsci494', 'stat491']
practice_class

['dsci494', 'stat491']

In [12]:
df.head()

,Org Defined ID,UserName,FirstName,LastName,Attempt #,Score,Out Of,Attempt_Start,Attempt_End,Percent,Class,Category,Module
0,14460432,au9747cp,Jericho,Greer,1,1,1,2019-01-14 14:00:00,2019-01-14 14:06:00,100 %,dsci494,Attendance,NaN
1,14460432,au9747cp,Jericho,Greer,2,1,1,2019-01-16 14:00:00,2019-01-16 14:08:00,100 %,dsci494,Attendance,NaN
2,14460432,au9747cp,Jericho,Greer,3,1,1,2019-01-18 14:00:00,2019-01-18 14:05:00,100 %,dsci494,Attendance,NaN
3,14460432,au9747cp,Jericho,Greer,4,1,1,2019-01-23 14:00:00,2019-01-23 14:06:00,100 %,dsci494,Attendance,NaN
4,14460432,au9747cp,Jericho,Greer,5,1,1,2019-01-25 14:00:00,2019-01-25 14:10:00,100 %,dsci494,Attendance,NaN


In [13]:
rename = {r:num for r,num in zip([1.0, 2.0, 3.0, 4.0],
                                ['Module_1','Module_2','Module_3','Module_4'])}
rename 

{1.0: 'Module_1', 2.0: 'Module_2', 3.0: 'Module_3', 4.0: 'Module_4'}

In [14]:
from more_dfply import ifelse
def practsum (classid, df):
    return (df
            >> select(X.FirstName, X.LastName, X.Module, X['Attempt #'], X.Class, X.Category)
            >> filter_by(X.Class == classid, X.Category == 'Practice')
            >> group_by(X.Class, X.FirstName, X.LastName, X.Module)
            >> summarize(Module_max = X['Attempt #'].max())
            >> mutate(TempMod = X.Module.map(rename))
            >> drop(X.Module)
            >> spread(X.TempMod, X.Module_max)
            >> mutate(Mod_1 = X.Module_1)
            >> mutate(Mod_2 = ifelse(X.Module_2.isna(), 0, X.Module_2))
            >> mutate(Mod_3 = ifelse(X.Module_3.isna(), 0, X.Module_3))
            >> mutate(Mod_4 = ifelse(X.Module_4.isna(), 0, X.Module_4))
            >> drop(X.Module_1, X.Module_2, X.Module_3, X.Module_4)
            >> group_by(X.FirstName, X.LastName)
            >> mutate(Total_Attempts = X.Mod_1 + X.Mod_2 + X.Mod_3 + X.Mod_4)
            #>> drop(X.Class)
           )
practsum('stat491', df)

,LastName,FirstName,Class,Mod_1,Mod_2,Mod_3,Mod_4,Total_Attempts
8,Leonardo,Appalachia,stat491,7.0,0.0,0.0,0.0,7.0
0,Acadia,Athenian,stat491,11.0,0.0,0.0,0.0,11.0
7,Italian,Bergland,stat491,14.0,0.0,0.0,0.0,14.0
3,Bogota,Dar,stat491,13.0,0.0,0.0,0.0,13.0
9,Louisa,Daytona,stat491,17.0,0.0,0.0,0.0,17.0
11,Nibelung,Dorset,stat491,26.0,0.0,0.0,0.0,26.0
4,Czech,Ekstrom,stat491,12.0,0.0,8.0,7.0,27.0
1,Angelica,Frau,stat491,10.0,5.0,17.0,7.0,39.0
6,Erich,Hammond,stat491,9.0,0.0,0.0,0.0,9.0
12,Odin,Hannah,stat491,18.0,0.0,6.0,0.0,24.0


In [15]:
col_names = ['LastName', 'FirstName', 'Class', 'Mod_1','Mod_2', 'Mod_3', 'Mod_4', 'Total_Attempts']
practicequiz = pd.DataFrame(columns=col_names)
for cls in practice_class:
    practicequiz = practicequiz >> union(practsum(cls, df)) 

In [16]:
practicequiz >> drop(X.Class) 

,LastName,FirstName,Mod_1,Mod_2,Mod_3,Mod_4,Total_Attempts
0,Aug,Ben,9,10,8,7,34
9,Montmartre,Christendom,13,11,15,8,47
1,Bali,Dickerson,10,15,8,16,49
2,Cargill,Goodyear,12,13,8,8,41
5,Greer,Jericho,17,12,9,8,46
7,Harbin,Mansfield,14,16,10,16,56
6,Hadrian,Merrimack,11,6,4,7,28
4,Gaylord,Philippine,11,11,7,10,39
3,Galbreath,Ross,17,8,8,7,40
8,Kafka,Willard,21,10,10,8,49


In [17]:
dsci494practice = practicequiz >> filter_by(X.Class == 'dsci494') >> drop(X.Class)
dsci494practice.to_csv('./data/attendance_example/dsci494s7/dsci494practice.csv', index=False)

In [18]:
stat491practice = practicequiz >> filter_by(X.Class == 'stat491') >> drop(X.Class)
stat491practice.to_csv('./data/attendance_example/stat491s1/stat491practice.csv', index=False)